- 학습과 테스트 중간에 '검증(validation)' 과정이 필요함 - 모델 최적화와 overfitting 방지를 위함.
- 교차검증(CV; Cross Validation). 가장 대표적인 K-Fold 를 주로 사용.
- cross_val_score() 라이브러리는 K-Fold 는 반복문 없이 간단히 수행. 분류문제는 알아서 Stratified K-Fold로.
- GridSearchCV() 라이브러리는 교차검증과 최적화(하이퍼파라미터 튜닝)을 한번에 할수있게 하는 API 

In [4]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target

iris_data = iris_df.drop(columns=['label'])
iris_label = iris_df['label']

x_train, x_test, y_train, y_test = train_test_split(iris_data, iris_label,
                                                   test_size = 0.2, random_state=11)

In [ ]:
# cross_val_score 라이브러리를 통한 K-Fold 교차검증과 결과 출력

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svm_model = SVC(kernel='rbf', C=1, gamma='scale')
cv_scores = cross_val_score(svm_model, x_train, y_train, cv=5, scoring='accuracy')

print(cv_scores, np.mean(cv_scores), np.std(cv_scores))

[0.91666667 0.91666667 1.         0.91666667 1.        ] 0.95 0.04082482904638632


In [13]:
# GridSearchCV 라이브러리를 통해 CV 와 최적화(튜닝)을 동시 진행
# (하이퍼 파라미터 조합개수 × CV횟수) 만틈의 평가를 진행 
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터는 딕셔너리 형태로..
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}

# GridSearch 는 객체를 만들고, fit 으로 실행함. 
grid_search = GridSearchCV(
    SVC(),  # SVM 분류기
    param_grid,  # 하이퍼파라미터 그리드
    cv=3,  # 5-Fold 교차검증
    scoring='accuracy',  # 평가 기준: 정확도 // 없으면 디폴트 값으로
    refit = True, # 최적의 파라미터 찾은 뒤 해당 파라미터로 재학습 (True가 디폴트값) 
    verbose=1,  # 진행 상황 출력
    n_jobs=-1  # CPU 병렬 처리 (최대 사용)
)
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=1)

In [14]:
print("최적의 하이퍼파라미터:", grid_search.best_params_)
print("최고 검증 정확도:", grid_search.best_score_)

results_df = pd.DataFrame(grid_search.cv_results_)[['params', 'split0_test_score',
                                                    'split1_test_score', 'split2_test_score',
                                                    'mean_test_score', 'rank_test_score']]

print("검증 결과:", results_df.head(2)) # 검증 결과에서 필요한 부분만 데이터프레임으로 표현

best_model = grid_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
print("최적의 하이퍼파라미터가 적용된 모델:", best_model)
print(test_accuracy)

최적의 하이퍼파라미터: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
최고 검증 정확도: 0.975
검증 결과:                                              params  split0_test_score  \
0  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}              0.900   
1     {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}              0.925   

   split1_test_score  split2_test_score  mean_test_score  rank_test_score  
0              0.975              0.975            0.950                8  
1              0.900              0.950            0.925               17  
최적의 하이퍼파라미터가 적용된 모델: SVC(C=1, kernel='linear')
1.0
